<img src="https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite@3x.png" alt="drawing" width="600"/>

# Wildfire Composit Risk Assessment



<table border="0">
 <tr>
         <td><b style="font-size:30px"><img style="display: block; margin: auto; width:500px" src="./assets/ca-fire.jpg"/></b></td>
    <td width="50%" style="font-size: 19px;">Today We Will:
1. Read Vector and Raster Data
2. Visualizing Data
4. Scaled Spatial Joins between vector and multiple raster layers
   

Data sets:
- [Burn Probability](!https://www.fs.usda.gov/rds/archive/catalog/RDS-2020-0016-2)
- Overture Maps
  - Buildings
  - Places (Fire Stations)
 
Process:
1. Define mapping units
2. Zonal Stats on Burn Probability
3. Calculate Building Densities
5. KNN to fire stations
6. Combine Metrics
7. Hot-Spot Detection
8. Profit


Along the way we'll introduce some important concepts for working with WherobotsDB like the Spatial DataFrame structure and querying WherobotsDB databases and tables using Spatial SQL.

Let's get started!
</td>

 </tr>
 <tr>
 </tr>
</table>


<table border="0" align='left' width="100%">


 <tr>
         <td width="75%px" style="font-size:22px">
        When using Apache Sedona (in Wherobots or not) the first step is to create the `Context`. This context represents the connection to the cluster. The first time we set the config and create the `Context` all of the nodes in the cluster are brought on line
    </td>
         <td align='right'>
             <b style="font-size:30px"><img align='right' style="display: block; margin: auto; width:250px" src="https://25.media.tumblr.com/ba7e79ff0c1fd792cd1fe2236e25f184/tumblr_mszn71jdjl1qmgfhzo1_1280.gif"/></b></td>
 </tr>
 <tr>
 </tr>
</table>

In [1]:
%%time
from sedona.spark import *
from pyspark.sql.functions import col, count, when, expr,lit,explode,broadcast, desc
from datetime import datetime
start = datetime.now()

config = SedonaContext.builder(). \
    config("spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider","org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider"). \
    getOrCreate()


sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


CPU times: user 480 ms, sys: 237 ms, total: 717 ms
Wall time: 1min 20s




<table border="0" align='left' width = "100%">


 <tr>
         <td width="44%" style="font-size:18px">
Our first layer to perpare is a "Burn Probability" Layer generaeted for by the USDA and Forest Service<sup>*</sup>. <br/> <br/> This raster represents "the annual probability of wildfire burning in a specific location." 
    </td>
     <td><img align='right' style="display: block; margin: auto; width:500px" src="https://wildfirerisk.org/wp-content/uploads/2024/05/thumbnail_BP_V2-scaled.jpg"/></td>
         <td align='right'>
             <b style="font-size:30px">
                 
<img align='right' style="display: block; margin: auto; width:250px" src="https://i.pinimg.com/originals/e3/78/be/e378be4bb4b16ca79b71e88c76fec900.gif"/>
             </b>
         </td>
 </tr>
 <tr>
     
    
 </tr>
</table>



In [2]:
%%time
analysis_start = datetime.now()
burn_probability = "s3://wherobots-examples/data/examples/SDSC/BP-clip-reblock-4326.tif"
exclusion_wkt = "MultiPolygon (((-119.22425369 31.67241078, -119.03742835 30.92881246, -117.00686077 31.26615639, -117.24927621 32.01643187, -117.36169794 32.50929768, -117.40697765 32.65986814, -117.45368807 32.79856211, -117.50463868 32.9832308, -117.59444643 33.13521645, -117.81002531 33.3313064, -118.16274878 33.51473174, -118.44751165 33.54860864, -118.54278981 33.70994242, -118.62402192 33.86683156, -118.77557516 33.93388227, -118.86933137 33.98250615, -118.89850234 33.97769324, -119.25883116 33.91883788, -119.84475305 33.82227912, -119.35342742 32.16682393, -119.28359676 31.88814932, -119.22425369 31.67241078)))"

burn_probability_df = sedona.sql(f"""
SELECT 
    'BP-clip-4326.tif' AS path, 
    RS_FromPath("{burn_probability}") AS rast""")
burn_probability_df.createOrReplaceTempView("burn_probability")

CPU times: user 1.81 ms, sys: 621 μs, total: 2.43 ms
Wall time: 93.5 ms


In [3]:
burn_probability_df_tiled = sedona.sql(f""" 
with exp_raster as (
SELECT 
    RS_TileExplode(rast,256,256) as (x, y, tile),
    path
from burn_probability ),

exp_ras_sub as(
select * from exp_raster
where  ST_INTERSECTS(RS_ENVELOPE(tile), ST_GEOMFROMWKT('{exclusion_wkt}')) = False)


SELECT
    *, 
    RS_SUMMARYSTATS(tile,'max') as max_bp,
    RS_SUMMARYSTATS(tile,'mean') as mean_bp, 
    RS_SUMMARYSTATS(tile,'min') as min_bp
from exp_ras_sub 
where  RS_SUMMARYSTATS(tile, 'count') > 0
"""
                                )
burn_probability_df_tiled.createOrReplaceTempView("burn_probability_tiled")

In [4]:
%%time
burn_probability_df_tiled.select("x", "y","mean_bp","max_bp","min_bp").show(truncate=False)

+---+---+---------------------+---------------------+---------------------+
|x  |y  |mean_bp              |max_bp               |min_bp               |
+---+---+---------------------+---------------------+---------------------+
|36 |0  |1.6749661990305068E-4|1.8135958816856146E-4|1.1338209878886119E-4|
|37 |0  |2.02409643301056E-5  |1.7960890545509756E-4|0.0                  |
|38 |0  |4.415770374645719E-5 |1.091641970560886E-4 |0.0                  |
|39 |0  |1.8770373336750017E-4|2.70210177404806E-4  |6.132179987616837E-5 |
|40 |0  |1.8849960687019061E-4|3.84111306630075E-4  |1.5960193195496686E-5|
|41 |0  |5.0818139004461404E-5|1.750174124026671E-4 |9.999999747378752E-6 |
|42 |0  |1.4710356230911117E-5|1.0809641389641911E-4|0.0                  |
|43 |0  |3.0346563184530973E-5|2.135593385901302E-4 |0.0                  |
|44 |0  |5.460342439923773E-5 |1.5832169447094202E-4|9.999999747378752E-6 |
|29 |1  |0.0015682515791720834|0.0018817137461155653|0.0010702286381274462|
|30 |1  |0.0

### Generate our mapping unit, H3 Level 7 hexagons

In [5]:
%%time
# 9.79 s
h3_level=6



hex_df = sedona.sql(f""" 
WITH h3_ids AS (SELECT 
   DISTINCT EXPLODE
               (
                   ST_H3CellIDs(RS_Envelope(tile), {h3_level}, true)
               ) AS h3 
FROM 
burn_probability_tiled),

exp_h3 as (
SELECT 
    h3,
    EXPLODE (ST_H3ToGeom(ARRAY(h3))) AS geometry
FROM 
    h3_ids)

select * from exp_h3 
where  ST_INTERSECTS(geometry, ST_GEOMFROMWKT('{exclusion_wkt}')) = False
""").repartition(15).cache()

hex_df.createOrReplaceTempView("mapping_units")
hex_df.count()



CPU times: user 27.1 ms, sys: 3.12 ms, total: 30.2 ms
Wall time: 1min 15s


1456

In [6]:
tiledMap = SedonaKepler.create_map()
SedonaKepler.add_df(tiledMap, hex_df, name="h3")
SedonaKepler.add_df(tiledMap, burn_probability_df_tiled.withColumn("tile", expr("RS_Envelope(tile)")).withColumn("area",expr("ST_AREA(tile)")), name="tiles")
tiledMap

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'h3': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, …

### Calculate Building Density



<table border="0" align='left' width = "100%">


 <tr>
         <td width="44%" style="font-size:18px">
Here we use the Overture Maps Foundation Buildings data set to calculate building density metric for our composit riskl score.  
    </td>
         <td align='right'>
             <b style="font-size:30px">
                 
<img align='right' style="display: block; margin: auto; width:350px" src="https://media.tenor.com/tjcVVIfgBcMAAAAC/city-cave.gif"/>
             </b>
         </td>
 </tr>
 <tr>
     
    
 </tr>
</table>


In [7]:
hex_df.printSchema()

root
 |-- h3: long (nullable = true)
 |-- geometry: geometry (nullable = true)



In [8]:
%%time

aoi_geom = sedona.sql("""select ST_Union_Aggr(geometry) as geom
    from mapping_units""").collect()[0].geom

CPU times: user 5.17 ms, sys: 0 ns, total: 5.17 ms
Wall time: 1.29 s


In [9]:
%%time
#Wall time: 10.3 s
aoi_buildings_df = sedona.sql(
f"""
select 
    ID, geometry
from 
    wherobots_open_data.overture_2024_05_16.buildings_building
where 
    ST_WITHIN(geometry,ST_GEOMFROMWKT('{aoi_geom}')) """)

CPU times: user 3 ms, sys: 0 ns, total: 3 ms
Wall time: 1.73 s


In [10]:
%%time
aoi_buildings_df = aoi_buildings_df.repartition(100)
aoi_buildings_df.createOrReplaceTempView('aoi_buildings')
aoi_buildings_df.count()

CPU times: user 12.5 ms, sys: 8.03 ms, total: 20.5 ms
Wall time: 19.9 s


8509401

In [11]:
%%time
building_density_df = sedona.sql(
"""
Select 
    t1.h3,
    t1.geometry,
    COUNT(t2.ID) building_count,
    (SUM(ST_AreaSpheroid(t2.geometry))/ST_AreaSpheroid(t1.geometry)) building_density
FROM 
    mapping_units t1
 JOIN
    aoi_buildings t2
ON
    ST_INTERSECTS(t1.geometry,ST_CENTROID(t2.geometry))
GROUP BY
    t1.h3,
    t1.geometry
""" 
)
building_density_df.createOrReplaceTempView("building_density")

CPU times: user 1.85 ms, sys: 0 ns, total: 1.85 ms
Wall time: 111 ms


In [12]:
%%time
#Wall time: 2min 24s
#Wall time: 39min 23s
building_density_df = building_density_df.cache()

building_density_df.show(20)

+------------------+--------------------+--------------+--------------------+
|                h3|            geometry|building_count|    building_density|
+------------------+--------------------+--------------+--------------------+
|604214651923726335|POLYGON ((-117.25...|         13717| 0.10129795743245161|
|604214645481275391|POLYGON ((-116.81...|          2366| 0.00877934829324488|
|604214731917492223|POLYGON ((-118.64...|          3578|0.018010992210975338|
|604214631119978495|POLYGON ((-117.40...|          3463| 0.02233795328393633|
|604214660782096383|POLYGON ((-117.52...|           235|0.001084597139425559|
|604214663063797759|POLYGON ((-117.47...|           279|0.001117218401442...|
|604214811105951743|POLYGON ((-116.75...|            12|5.605720815404745E-5|
|604214633535897599|POLYGON ((-117.55...|            11|3.820247750130015...|
|604214903716184063|POLYGON ((-117.31...|             9|6.078803896774047E-5|
|604215093097398271|POLYGON ((-115.67...|             2|2.311848

In [13]:
%%time
#Wall time: 1min 1s
max_burn_df = sedona.sql(f"""

SELECT
    t1.h3,
    t1.geometry,
    t1.building_count,
    t1.building_density,
    MAX(t2.max_bp) as max_bp, 
    MIN(t2.min_bp) as min_bp,
    AVG(t2.mean_bp) as mean_bp,
    MAX(t2.max_bp) - MIN(t2.min_bp) as range_bp

from 
    building_density t1
 join
  burn_probability_tiled t2
on ST_INTERSECTS(t1.geometry,RS_ENVELOPE(t2.tile))
group by 
1,2,3,4


""")
max_burn_df = max_burn_df.cache()
max_burn_df.createOrReplaceTempView("max_burn")
max_burn_df.show()

+------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                h3|            geometry|building_count|    building_density|              max_bp|              min_bp|             mean_bp|            range_bp|
+------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|604214651923726335|POLYGON ((-117.25...|         13717| 0.10129795743245161|0.002469598548486829|                 0.0|5.739324459825393E-4|0.002469598548486829|
|604214645481275391|POLYGON ((-116.81...|          2366| 0.00877934829324488| 0.04732268303632736|                 0.0|0.009512943033608173| 0.04732268303632736|
|604214731917492223|POLYGON ((-118.64...|          3578|0.018010992210975338|0.025503288954496384|                 0.0|0.003979053346991...|0.025503288954496384|
|604214631119978495|POLYGON 

In [14]:
building_density_df.unpersist()

DataFrame[h3: bigint, geometry: udt, building_count: bigint, building_density: double]

In [15]:
max_burn_df.printSchema()

root
 |-- h3: long (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- building_count: long (nullable = false)
 |-- building_density: double (nullable = true)
 |-- max_bp: double (nullable = true)
 |-- min_bp: double (nullable = true)
 |-- mean_bp: double (nullable = true)
 |-- range_bp: double (nullable = true)



### Let's prepare for a KNN join! 
<table border="0" align='left' width = "100%">
 <tr>
    <td width="50%" align="left" style="font-size: 19px;">We extract all the fire stations by primary catagory and using our AOI for spatial filter pushdown. 
</td>
     <td align="right"><b style="font-size:30px"><img align='right' style="display: block; margin: auto; width:500px" src="./assets/fireman.gif"/></b></td>
 

 </tr>

</table>

In [16]:
%%time
fire_stateion_df = sedona.sql(f"""
select * from 
    wherobots_open_data.overture_2024_07_22.places_place  
WHERE 
    categories.primary in ('fire_department')
AND
    ST_WITHIN(geometry,ST_GEOMFROMWKT('{aoi_geom}'))
""")
fire_stateion_df.count()
fire_stateion_df.createOrReplaceTempView("fire_stations")

CPU times: user 4.81 ms, sys: 1.08 ms, total: 5.89 ms
Wall time: 2.69 s


In [17]:
%%time
df_knn_join = sedona.sql("""
SELECT
    source.GEOMETRY AS SOURCE_GEOM,
    SOURCE.h3 AS h3,
    sink.GEOMETRY AS sink_GEOM,
    ST_DISTANCESPHERE(source.GEOMETRY, sink.GEOMETRY) AS DISTANCE,
    ST_MAKELINE(ST_CENTROID(source.GEOMETRY), sink.GEOMETRY) AS LINE
FROM max_burn source

JOIN fire_stations sink ON ST_AKNN(source.GEOMETRY, sink.GEOMETRY, 1, FALSE)
""")


CPU times: user 2.06 ms, sys: 0 ns, total: 2.06 ms
Wall time: 55.7 ms


In [18]:
%%time
# Wall time: 1min 56s
# Select N unique QID rows
df_unique_qid = df_knn_join.dropDuplicates(["SOURCE_GEOM"])

# Perform an inner join to get all rows from join_df that have QIDs in unique_qid_df
df_related_rows = df_knn_join.join(df_unique_qid, on="h3", how="inner").select(df_knn_join["*"])

df_unique_qid.cache()
df_related_rows.cache()
df_related_rows.createOrReplaceTempView("fire_station_distance")
df_related_rows.count()

24/10/16 14:10:32 WARN JoinQueryDetector: Warning: One or more filter pushdowns have been detected on the object side of the KNN join. 
These filters will be applied to the object side reader before the KNN join is executed. 
If you intend to apply the filters after the KNN join, please ensure that you materialize the KNN join results before applying the filters. 
For example, you can use the following approach:

Scala Example:
val knnResult = knnJoinDF.cache()
val filteredResult = knnResult.filter(condition)

SQL Example:
CREATE OR REPLACE TEMP VIEW knnResult AS
SELECT * FROM (
  -- Your KNN join SQL here
) AS knnView
CACHE TABLE knnResult;
SELECT * FROM knnResult WHERE condition;
24/10/16 14:10:32 WARN JoinQueryDetector: Warning: One or more filter pushdowns have been detected on the object side of the KNN join. 
These filters will be applied to the object side reader before the KNN join is executed. 
If you intend to apply the filters after the KNN join, please ensure that you mater

These filters will be applied to the object side reader before the KNN join is executed. 
If you intend to apply the filters after the KNN join, please ensure that you materialize the KNN join results before applying the filters. 
For example, you can use the following approach:

Scala Example:
val knnResult = knnJoinDF.cache()
val filteredResult = knnResult.filter(condition)

SQL Example:
CREATE OR REPLACE TEMP VIEW knnResult AS
SELECT * FROM (
  -- Your KNN join SQL here
) AS knnView
CACHE TABLE knnResult;
SELECT * FROM knnResult WHERE condition;
These filters will be applied to the object side reader before the KNN join is executed. 
If you intend to apply the filters after the KNN join, please ensure that you materialize the KNN join results before applying the filters. 
For example, you can use the following approach:

Scala Example:
val knnResult = knnJoinDF.cache()
val filteredResult = knnResult.filter(condition)

SQL Example:
CREATE OR REPLACE TEMP VIEW knnResult AS
SELECT * FR

CPU times: user 113 ms, sys: 16.6 ms, total: 130 ms
Wall time: 1min 56s


1101

In [19]:
df_unique_qid.show()

+--------------------+------------------+--------------------+------------------+--------------------+
|         SOURCE_GEOM|                h3|           sink_GEOM|          DISTANCE|                LINE|
+--------------------+------------------+--------------------+------------------+--------------------+
|POLYGON ((-118.08...|604214749902667775|POINT (-118.05825...| 3102.002678648912|LINESTRING (-118....|
|POLYGON ((-117.01...|604214678364618751|POINT (-117.02257...|  5182.15009966782|LINESTRING (-117....|
|POLYGON ((-118.53...|604214731514839039|POINT (-118.53636...|1756.1248502033416|LINESTRING (-118....|
|POLYGON ((-116.01...|604755134769004543|POINT (-115.99441...|15959.845548939582|LINESTRING (-116....|
|POLYGON ((-117.79...|604214672324820991|POINT (-117.76659...|3490.0959712307554|LINESTRING (-117....|
|POLYGON ((-118.54...|604214728427831295|POINT (-118.59157...|2855.3138102217645|LINESTRING (-118....|
|POLYGON ((-117.15...|604214656889782271|POINT (-117.18965...|  992.95465

In [20]:
# create map for the results
map_view = SedonaKepler.create_map(df_unique_qid.select('SOURCE_GEOM'), name="SOURCE")
SedonaKepler.add_df(map_view, df=df_related_rows.select('SINK_GEOM', 'DISTANCE').withColumnRenamed("SINK_GEOM", "geometry"), name="SINK")
SedonaKepler.add_df(map_view, df=df_related_rows.select('LINE', 'DISTANCE').withColumnRenamed("LINE", "geometry"), name="KNN LINES")

# show the map
map_view

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'SOURCE': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …

### Bringing it all together

With all of out risk factors collected, we can now weight them and generate our final risk score.

Feel free to play around with different weights! Just make sure they sum to `1` :) 

In [21]:
%%time
weights = {
"norm_building_count" : .15,
"norm_building_density" : .2,
"norm_max_bp" : .2,
"norm_min_bp" : .05,
"norm_range_bp" : .1,
"norm_mean_bp" : .1,
"from_distance_ff" : .2
}

norm_df = sedona.sql(f"""
with max_burn_ff_dist AS (

    SELECT 
        t1.*,
        t2.DISTANCE
    FROM 
        max_burn t1
    JOIN
        fire_station_distance t2
    USING (h3)

),


 min_max AS (
    SELECT
       MIN(building_count) AS min_building_count,
        MAX(building_count) AS max_building_count,
        MIN(building_density) AS min_building_density,
        MAX(building_density) AS max_building_density,
        MIN(max_bp) AS min_max_bp,
        MAX(max_bp) AS max_max_bp,
        MIN(min_bp) AS min_min_bp,
        MAX(min_bp) AS max_min_bp,
        MIN(range_bp) AS min_range_bp,
        MAX(range_bp) AS max_range_bp,
        MIN(mean_bp) AS min_mean_bp,
        MAX(mean_bp) AS max_mean_bp,
        MIN(DISTANCE) AS min_dist_ff,
        MAX(DISTANCE) AS max_dist_ff
    FROM 
        max_burn_ff_dist),

normalized_data AS (
    SELECT
        t1.h3,
        t1.geometry,
        (t1.building_count - s.min_building_count) / (s.max_building_count - s.min_building_count) AS norm_building_count,
        (t1.building_density - s.min_building_density) /  (s.max_building_density - s.min_building_density) AS norm_building_density,
        (t1.max_bp - s.min_max_bp) /     (s.max_max_bp - s.min_max_bp) AS norm_max_bp,
        (t1.min_bp - s.min_min_bp) /     (s.max_min_bp - s.min_min_bp) AS norm_min_bp,
        (t1.range_bp - s.min_range_bp) / (s.max_range_bp - s.min_range_bp) AS norm_range_bp,
        (t1.mean_bp - s.min_mean_bp) /  (s.max_mean_bp - s.min_mean_bp) AS norm_mean_bp,
        (t1.DISTANCE - s.min_dist_ff) /  (s.max_dist_ff - s.min_dist_ff) AS norm_distance_bp
    FROM
        max_burn_ff_dist t1, MIN_MAX s
        )
-- Now add the weights to the normalized columns
SELECT
    h3,
    geometry,
    norm_building_count * {weights['norm_building_count']} AS weighted_norm_building_count,
    COALESCE(norm_building_density,0) * {weights['norm_building_density']}  AS weighted_norm_building_density,
    norm_max_bp * {weights['norm_max_bp']}  AS weighted_norm_max_bp,
    norm_min_bp * {weights['norm_min_bp']}  AS weighted_norm_min_bp,
    norm_range_bp * {weights['norm_range_bp']}  AS weighted_norm_range_bp,
    norm_mean_bp * {weights['norm_mean_bp']}  AS weighted_norm_mean_bp,
    norm_distance_bp * {weights['from_distance_ff']}  AS weighted_norm_distance_bp,
    -- Optional: Aggregate all weighted scores for a final score if needed
    (norm_building_count * {weights['norm_building_count']}  + 
     norm_building_density * {weights['norm_building_density']}  + 
     norm_max_bp * {weights['norm_max_bp']}  + 
     norm_min_bp * {weights['norm_min_bp']}  + 
     norm_range_bp * {weights['norm_range_bp']} +
     norm_mean_bp * {weights['norm_mean_bp']} +
     norm_distance_bp * {weights['from_distance_ff']} ) AS total_weighted_score
FROM normalized_data

""")
norm_df =norm_df.cache()
norm_df.select("h3","total_weighted_score").show()

+------------------+--------------------+
|                h3|total_weighted_score|
+------------------+--------------------+
|604214651923726335| 0.08216583320578169|
|604214645481275391| 0.16064181315266648|
|604214631119978495| 0.07012467786284654|
|604214660782096383| 0.28642418371014333|
|604214663063797759| 0.35530279402834447|
|604214731917492223| 0.09603946145460544|
|604214811105951743| 0.06755859444723362|
|604214633535897599| 0.21017418002315683|
|604214903716184063|0.011225757259944888|
|604215093097398271|0.029831229789847796|
|604214681854279679| 0.17284410206988604|
|604214646823452671| 0.16110854422092963|
|604214751110627327| 0.24407306224404024|
|604755112891514879| 0.20769744427144007|
|604215084507463679| 0.22979832134289183|
|604214763861311487| 0.09017658316789635|
|604214631522631679| 0.10510273724979222|
|604755285763948543| 0.02176872057708523|
|604755114636345343|0.038845116144162244|
|604214931365036031| 0.10462243966166822|
+------------------+--------------

In [22]:
max_burn_df.unpersist()

DataFrame[h3: bigint, geometry: udt, building_count: bigint, building_density: double, max_bp: double, min_bp: double, mean_bp: double, range_bp: double]

### Make a map of the normalized risk factors

Create a choropleth map of the differetn risk factors and final risk score. Do any of the engineered features show a pattern?

In [23]:
tiledMap = SedonaKepler.create_map()
SedonaKepler.add_df(tiledMap, norm_df, name="Weighted Score")
tiledMap

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Weighted Score': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, …

In [24]:
norm_df.printSchema()

root
 |-- h3: long (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- weighted_norm_building_count: double (nullable = true)
 |-- weighted_norm_building_density: double (nullable = false)
 |-- weighted_norm_max_bp: double (nullable = true)
 |-- weighted_norm_min_bp: double (nullable = true)
 |-- weighted_norm_range_bp: double (nullable = true)
 |-- weighted_norm_mean_bp: double (nullable = true)
 |-- weighted_norm_distance_bp: double (nullable = true)
 |-- total_weighted_score: double (nullable = true)



### Examine Risk factors for Clusters

Getis-Ord  local Hot Spot analysis

https://docs.wherobots.com/latest/tutorials/wherobotsai/hotspot-detection/getis_ord/?h=geti

In [28]:
%%time
#Wall time: 9.39 s
from sedona.stats.hotspot_detection.getis_ord import g_local
from sedona.stats.weighting import add_binary_distance_band_column
from sedona.stats.weighting import add_distance_band_column



distance_band_radius = .01

gi_df = g_local(
    add_distance_band_column( # adds the "weights" columns
        norm_df,
        distance_band_radius,
        binary=True,
        include_self=True,
        self_weight=1.0
    ),
    "weighted_norm_distance_bp", ## <--------- TEST DIFFERENT RISK FACTORS HERE
    "weights",
    star=True
).drop("weights")\
.drop("h3CellGeom")
gi_df.show()

+------------------+--------------------+----------------------------+------------------------------+--------------------+--------------------+----------------------+---------------------+-------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
|                h3|            geometry|weighted_norm_building_count|weighted_norm_building_density|weighted_norm_max_bp|weighted_norm_min_bp|weighted_norm_range_bp|weighted_norm_mean_bp|weighted_norm_distance_bp|total_weighted_score|                   G|                  EG|                  VG|                   Z|                  P|
+------------------+--------------------+----------------------------+------------------------------+--------------------+--------------------+----------------------+---------------------+-------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-

#### Make a map and symbolize by Z value and the P score

In [29]:
tiledMap = SedonaKepler.create_map()
SedonaKepler.add_df(tiledMap, gi_df, name="Weighted Score")
tiledMap

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Weighted Score': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, …

In [27]:
print(f"Context Start Time: {(analysis_start-start)/60}")
print(f"Analysis Time: {(datetime.now()-analysis_start)/60}")
print(f"Total Time: {((analysis_start-start)/60) + ((datetime.now()-analysis_start)/60)}")


Context Start Time: 0:00:01.335381
Analysis Time: 0:00:08.519688


NameError: name 'analysis_time' is not defined